### Лабораторная работа №3 часть 2
Доскоч Роман 

In [1]:
import math
import pandas as pd
import numpy as np
import plotly

import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots
from numpy import sign, sqrt

from sklearn.model_selection import train_test_split

In [2]:
data = pd.read_csv(r'3-2_data.csv')
train, test =  train_test_split(data, test_size=0.3, random_state=2)

In [3]:
def mse(w1, w2, x=train.x, y=train.y):
    return sum((y - (w1*x+w2))**2)/len(x)

In [4]:
#global epsilon
g_e = .0001

f=mse
max_count=1000

def df_y(x, e, _x, _y):
    return (f(x, e + g_e, _x, _y) - f(x, e, _x, _y)) / g_e

def df_x(e, y, _x, _y):
    return (f(e + g_e, y, _x, _y) - f(e,y, _x, _y)) / g_e

def grad_descent_fast(x, y, _x=train.x, _y=train.y):
    path = [np.array([x, y, f(x, y, _x, _y)])]
    lr=1.3
    count = 0
    prev = 1
    while count< max_count and abs(f(x, y, _x, _y) - prev) > g_e:
        count+=1
        prev = f(x, y)
        lr *= .9
        x -= lr*sign(df_x(x, y, _x, _y))
        y -= lr*sign(df_y(x, y, _x, _y))
        path.append(np.array([x, y, f(x, y, _x, _y)]))
      
    np_path = np.array(path)
    return np_path[:,0], np_path[:,1], np_path[:,2]

def grad_descent(x, y, _x=train.x, _y=train.y):
    path = [np.array([x, y, f(x, y, _x, _y)])]
    prev = f(-x,-y, _x, _y)
    lr=0.01
    count = 0
    while count < max_count and abs(f(x,y, _x, _y) - prev) > g_e:
        count += 1
        prev = f(x, y, _x, _y)
        
        x -= lr*df_x(x, y, _x, _y)
        y -= lr*df_y(x, y, _x, _y)
        
        path.append(np.array([x, y, f(x, y, _x, _y)]))
      
    np_path = np.array(path)
    return np_path[:,0], np_path[:,1], np_path[:,2]

In [5]:
class SimpleLinearRegression:    
    def fit(self, x, y):
        self.x = x
        self.y = y
    
    def predict(self, x, s_w1, s_w2):
        X, Y, _ = grad_descent(s_w1, s_w2, self.x, self.y)
        return X[-1]*x + Y[-1]

In [10]:
n_points = 100
x_range = (-2, 2)
y_range = (-2, 4)

X = np.linspace(x_range[0], x_range[1], n_points)
Y = np.linspace(y_range[0], y_range[1], n_points)

Z = np.zeros((n_points, n_points))
for i, x in enumerate(X):
    for j, y in enumerate(Y):
        print(mse(x, y, train.x, train.y))
        Z[j][i] = mse(x, y, train.x, train.y)

w1, w2 = 1, 1
_X, _Y, _Z = grad_descent(w1, w2) 
__X, __Y, __Z = grad_descent_fast(w1, w2) 
line_trace = go.Scatter3d(x=_X,y=_Y,z=_Z, mode='lines+markers', marker=go.scatter3d.Marker(size=4.5))
line_trace_fast = go.Scatter3d(x=__X,y=__Y,z=__Z, mode='lines+markers', marker=go.scatter3d.Marker(size=4.5))
_data = [go.Surface(z=Z, x=X, y=Y), line_trace, line_trace_fast ]

fig = go.Figure(data=_data)
fig.update_layout(title='f(x,y) 3D plot', autosize=False, width=1000, height=1000)
fig.show()

60.213585555193276
59.32790095358891
58.4495625411489
57.57857031787325
56.71492428376201
55.85862443881513
55.00967078303261
54.16806331641443
53.3338020389607
52.506886950671266
51.68731805154624
50.875095341585585
50.07021882078929
49.27268848915736
48.48250434668981
47.69966639338663
46.92417462924783
46.15602905427338
45.39522966846332
44.64177647181762
43.895669464336336
43.15690864601936
42.42549401686678
41.70142557687856
40.984703326054714
40.275327264395266
39.57329739190017
38.87861370856945
38.191276214403075
37.511284909401105
36.8386397935635
36.17334086689024
35.515388129381364
34.86478158103688
34.22152122185675
33.58560705184098
32.957039070989616
32.33581727930259
31.721941676779938
31.115412263421657
30.51622903922775
29.92439200419823
29.33990115833306
28.762756501632268
28.19295803409585
27.630505755723807
27.075399666516113
26.527639766472813
25.98722605559388
25.45415853387932
24.92843720132912
24.410062057943314
23.899033103721845
23.39535033866477
22.8990137627

17.532488172925817
17.118899580607522
16.712657177453618
16.313760963464077
15.922210938638923
15.538007102978112
15.161149456481693
14.791637999149645
14.429472730981963
14.074653651978643
13.727180762139701
13.387054061465125
13.054273549954933
12.728839227609104
12.41075109442765
12.100009150410566
11.79661339555785
11.500563829869494
11.211860453345528
10.930503265985928
10.656492267790695
10.38982745875983
10.130508838893348
9.878536408191223
9.633910166653484
9.3966301142801
9.166696251071098
8.944108577026467
8.728867092146201
8.520971796430306
8.320422689878786
8.127219772491635
7.941363044268853
7.7628525052104465
7.591688155316409
7.427869994586742
7.271398023021446
7.122272240620523
6.980492647383969
6.846059243311789
6.718972028403973
6.5992310026605345
6.486836166081464
6.381787518666767
6.284085060416438
6.193728791330479
6.110718711408893
6.03505482065168
5.966737119058835
5.9057656066303625
5.85214028336626
5.805861149266531
5.766928204331171
5.735341448560182
5.7111008

13.186381901130357
12.862887481550878
12.546739251135772
12.237937209885036
11.936481357798666
11.642371694876672
11.35560822111905
11.076190936525798
10.804119841096915
10.539394934832403
10.28201621773226
10.03198368979649
9.789297351025093
9.553957201418065
9.325963240975407
9.105315469697125
8.892013887583207
8.68605849463367
8.487449290848497
8.296186276227692
8.112269450771265
7.935698814479204
7.766474367351512
7.604596109388193
7.45006404058925
7.302878160954674
7.163038470484466
7.030544969178633
6.905397657037171
6.78759653406008
6.67714160024736
6.574032855599012
6.478270300115032
6.389853933795425
6.308783756640189
6.235059768649321
6.168681969822828
6.109650360160702
6.05796493966295
6.013625708329565
5.97663266616056
5.946985813155915
5.924685149315647
5.909730674639751
5.902122389128225
5.901860292781065
5.908944385598285
5.923374667579868
5.945151138725827
5.974273799036154
6.010742648510854
6.054557687149922
6.105718914953364
6.164226331921175
6.230079938053359
6.30327

9.971190435959008
9.767174945775812
9.570505644756992
9.381182532902537
9.199205610212454
9.024574876686748
8.857290332325404
8.69735197712844
8.544759811095837
8.399513834227614
8.261614046523757
8.131060447984273
8.00785303860916
7.89199181839842
7.783476787352046
7.6823079454700425
7.588485292752411
7.502008829199157
7.422878554810266
7.3510944695857505
7.286656573525603
7.229564866629831
7.179819348898428
7.137420020331393
7.102366880928732
7.07465993069044
7.054299169616519
7.041284597706973
7.035616214961795
7.03729402138099
7.046318016964548
7.062688201712489
7.086404575624791
7.1174671387014685
7.155875890942516
7.201630832347938
7.254731962917727
7.315179282651892
7.382972791550419
7.458112489613323
7.540598376840595
7.630430453232241
7.7276087187882565
7.832133173508641
7.944003817393403
8.063220650442531
8.189783672656032
8.323692884033902
8.464948284576144
8.613549874282752
8.769497653153744
8.932791621189093
9.10343177838882
9.281418124752912
9.46675066028138
9.65942938497

12.392586479600396
12.175818513854807
11.966396737273593
11.764321149856746
11.569591751604277
11.382208542516167
11.20217152259244
11.02948069183307
10.864136050238088
10.706137597807464
10.555485334541219
10.412179260439343
10.27621937550183
10.147605679728704
10.026338173119932
9.912416855675538
9.805841727395517
9.706612788279866
9.61473003832859
9.53019347754167
9.453003105919137
9.38315892346097
9.320660930167172
9.265509126037744
9.217703511072694
9.177244085272008
9.1441308486357
9.118363801163754
9.099942942856185
9.08886827371299
9.085139793734157
9.088757502919695
9.099721401269608
9.11803148878389
9.143687765462545
9.176690231305571
9.217038886312968
9.264733730484739
9.31977476382088
9.382161986321387
9.45189539798627
9.528974998815523
9.613400788809146
9.705172767967134
9.804290936289501
9.910755293776237
10.024565840427345
10.145722576242825
10.27422550122267
10.41007461536689
10.553269918675486
10.703811411148452
10.861699092785775
11.026932963587482
11.199513023553557


...

220.97808094460413
222.5111794780364
224.05162420063297
225.59941511239398
227.15455221331928
228.71703550340897
230.28686498266316
231.8640406510814
233.4485625086643
235.04043055541146
236.639644791323
238.24620521639898
239.8601118306392
241.48136463404387
243.1099636266129
244.74590880834626
246.38920017924397
248.03983773930617
249.69782148853264
251.36315142692354
253.03582755447874
254.71584987119843
256.40321837708245
258.09793307213084
259.7999939563436
261.50940102972066
263.22615429226215
264.9502537439679
266.6816993848383
268.42049121487287
270.1666292340718
271.92011344243514
273.68094383996294
140.41938315442553
141.4863219930479
142.56060702083468
143.6422382377858
144.7312156439013
145.8275392391811
146.93120902362543
148.042224997234
149.1605871600069
150.28629551194427
151.41935005304603
152.55975078331207
153.70749770274264
154.86259081133736
156.02503010909652
157.19481559602008
158.37194727210806
159.55642513736035
160.7482491917771
161.9474194353582
163.153935868

193.30765724744438
194.64434857620492
195.9883860941299
197.3397698012192
198.6984996974728
200.06457578289084
201.4379980574732
202.81876652121994
204.2068811741311
205.60234201620662
207.00514904744654
208.41530226785085
209.8328016774194
211.25764727615243
212.68983906404975
214.12937704111147
215.57626120733758
217.030491562728
218.49206810728285
219.96099084100211
221.43725976388572
222.92087487593358
224.41183617714603
225.91014366752268
227.41579734706374
228.92879721576918
230.44914327363904
231.97683552067323
233.51187395687174
235.05425858223467
236.60398939676188
238.16106640045365
239.72548959330967
241.2972589753301
242.87637454651497
244.46283630686418
246.05664425637755
247.65779839505552
249.26629872289763
250.8821452399044
252.50533794607546
254.13587684141072
255.77376192591046
257.41899319957474
259.0715706624031
260.73149431439595
262.3987641555532
264.0733801858748
265.7553424053607
267.4446508140111
269.1413054118259
270.8453061988048
272.55665317494834
274.275346

313.6284682816354
315.4773363429584
317.33355059344564
319.19711103309726
321.0680176619133
322.9462704798938
324.83186948703866
326.72481468334774
328.62510606882137
330.5327436434592
332.44772740726165
334.37005736022826
336.2997335023592
338.23675583365457
340.18112435411433
342.13283906373846
344.091899962527
346.05830705048
348.0320603275972
350.0131597938788
352.0016054493248
353.997397293935
356.00053532770977
358.01101955064905
360.0288499627526
362.0540265640203
364.0865493544525
366.1264183340492
368.17363350280993
207.85467285397888
209.20799646383443
210.56866626285452
211.93668225103897
213.31204442838757
214.6947527949007
216.08480735057816
217.48220809542008
218.88695502942628
220.29904815259692
221.71848746493183
223.14527296643118
224.5794046570949
226.02088253692304
227.46970660591546
228.92587686407222
230.38939331139352
231.86025594787898
233.33846477352904
234.8240197883433
236.31692099232188
237.817168385465
239.3247619677724
240.8397017392441
242.36198769988047
2

271.0419896335923
272.65771954442164
274.2807956444155
275.91121793357365
277.54898641189624
279.1941010793832
280.84656193603456
282.50636898185013
284.17352221683007
285.8480216409745
287.52986725428326
289.21905905675646
290.91559704839386
292.6194812291956
294.3307115991619
296.04928815829254
297.7752109065875
299.50847984404703
301.2490949706706
302.99705628645876
304.7523637914113
306.51501748552795
308.28501736880935
310.06236344125494
311.8470557028649
313.63909415363935
315.438478793578
317.245209622681
319.05928664094847
320.88070984838015
322.70947924497625
324.545594830737
326.38905660566195
328.2398645697511
330.0980187230049
331.96351906542293
333.8363655970053
335.7165583177521
337.6040972276632
339.4989823267388
341.4012136149787
343.310791092383
345.2277147589517
347.1519846146845
349.08360065958203
351.0225628936438
352.96887131686975
354.9225259292604
356.88352673081516
358.8518737215345
360.8275669014181
362.81060627046605
364.80099182867843
366.7987235760551
368.80

399.60292409544303
401.6867536038489
403.7779293014188
405.876451188153
407.9823192640516
410.09553352911496
412.2160939833424
414.34400062673416
416.47925345929036
418.62185248101105
420.7717976918959
422.9290890919451
425.0937266811586
427.26571045953705
429.44504042707933
431.6317165837863
433.8257389296571
436.02710746469296
438.2358221888928
440.45188310225706
442.6752902047858
444.9060434964788
447.1441429773362
449.38958864735775
451.6423805065441
453.90251855489475
456.1700027924096
458.444833219089
460.7270098349324
463.01653263994035
465.3134016341126
467.61761681744935
469.92917818995056
472.248085751616
474.5743395024459
476.90793944243984
289.53157883031497
291.17128721140386
292.8183417816572
294.47274254107487
296.13448948965674
297.803582627403
299.4800219543138
301.16380747038903
302.85493917562854
304.55341707003237
306.2592411536005
307.97241142633294
309.69292788823014
311.4207905392914
313.1559993795172
314.8985544089075
316.6484556274616
318.4057030351806
320.1702

In [11]:
w1,w2=_X[-1],_Y[-1]
print(f'{w1=},{w2=}')

w1=-1.6601832873347497,w2=3.4818029576988643


Попробуйте отключить все модификации градиентного спуска. И инициализировать learning rate(коэффициент при антиградиенте) не маленьким значением(как например 10e-4), а чем-то типа единицы или больше. Как отработал алгоритм? Если что-то пошло не так(должно пойти), то что и почему? 

<hr style="border: 2px solid black">
при значении learning rate = 0.9 значения функции привысили $10^{26}$<br>
Причина этому возможность вылета из области минимума, и никогда больше не вернуться в начальную точку

In [8]:
fig = go.Figure()
x=np.linspace(min(data['x']),max(data['x']),100)
fig.add_trace(go.Scatter(x=x, y=w1*x + w2))
fig.add_trace(go.Scatter(x=x, y=-1.7758*x+4.5338))
fig.add_trace(go.Scatter(x=train.x, y=train.y, mode='markers', marker=dict(color='green')))
fig.add_trace(go.Scatter(x=test.x, y=test.y, mode='markers', marker=dict(color='blue')))

In [9]:
from sklearn.metrics import r2_score

lin_reg = SimpleLinearRegression()
lin_reg.fit(train.x, train.y)
pred_y = lin_reg.predict(test.x,1,1)
real_y = test.y

RMSE = sqrt(mse(w1,w2))
print(f'r2_score={r2_score(real_y, pred_y)}\n{RMSE=}')
print(f'{mse(w1,w2)}')

r2_score=0.7583534570931587
RMSE=2.379615033837117
5.662567709263622
